# Tinkering With with APPG Data

In [125]:
import sqlite3
import pandas as pd


# Available tables
'''
df_AGM
df_contacts
df_details
df_inKindBenefits
df_officers
df_regBenefits
'''

conn = sqlite3.connect('appg-data.db')
query = """SELECT * FROM df_inKindBenefits"""

df = pd.read_sql_query(query,conn)
df.head()

,Source,Description,"Value £s In bands of £1,500",Received,Registered,gid
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan
1,Royal African Society,Secretariat From : 08/01/2021 To : 07/01/2022,"31,501-33,000",08/01/2021,21/04/2021,africa
2,Chatham House,Secretariat From : 29/01/2021 To : 28/01/2022,"9,001-10,500",29/01/2021,08/03/2021,angola
3,CalComms,Secretariat From : 10/09/2020 To : 09/09/2021,"13,501-15,000",10/09/2020,24/09/2020,canada-australia-new-zealand-and-the-uk-canzuk
4,Samir Takla,Samir Takla acts as the group's secretariat an...,"4,501-6,000",29/01/2021,15/02/2021,egypt


## Pattern Based Extraction

If there are regular text patterns in a column, we can create a function parse that elements out of it. For example, if the secretariat is *always* described the same way, we can pull that out:

In [98]:
from parse import parse

fromto = lambda txt: parse('{thing}From : {from:tg}  To : {to:tg}', txt)

secretariat = fromto("Secretariat  From : 15/09/2020  To : 14/09/2021")
secretariat['thing'], secretariat['from'], secretariat['to']

('Secretariat  ',
 datetime.datetime(2020, 9, 15, 0, 0),
 datetime.datetime(2021, 9, 14, 0, 0))

Apply the secretariat period extractor to each row, putting the results into the `from_to` column:

In [99]:
#parse('{}From : {from:tg} To : {to:tg}', "Secretariat From : 29/01/2021 To : 28/01/2022")
df['from_to'] = df["Description"].apply(fromto)

df.head()

,Source,Description,"Value £s In bands of £1,500",Received,Registered,gid,from_to
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"<Result () {'thing': 'Secretariat ', 'from': ..."
1,Royal African Society,Secretariat From : 08/01/2021 To : 07/01/2022,"31,501-33,000",08/01/2021,21/04/2021,africa,"<Result () {'thing': 'Secretariat ', 'from': ..."
2,Chatham House,Secretariat From : 29/01/2021 To : 28/01/2022,"9,001-10,500",29/01/2021,08/03/2021,angola,"<Result () {'thing': 'Secretariat ', 'from': ..."
3,CalComms,Secretariat From : 10/09/2020 To : 09/09/2021,"13,501-15,000",10/09/2020,24/09/2020,canada-australia-new-zealand-and-the-uk-canzuk,"<Result () {'thing': 'Secretariat ', 'from': ..."
4,Samir Takla,Samir Takla acts as the group's secretariat an...,"4,501-6,000",29/01/2021,15/02/2021,egypt,"<Result () {'thing': ""Samir Takla acts as the ..."


We can perhaps more conveniently pull those elements into their own columns:

In [102]:
from pandas import Series

explode_from_to = lambda x: (x['from_to']['thing'], x['from_to']['from'], x['from_to']['to'])

df2 = pd.merge(df, df.dropna().apply(explode_from_to, axis=1).apply(Series), left_index=True, right_index=True)
df2.rename(columns={0:"thing", 1:"from", 2:"to"}, inplace=True)
df2.head()

,Source,Description,"Value £s In bands of £1,500",Received,Registered,gid,from_to,thing,from,to
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"<Result () {'thing': 'Secretariat ', 'from': ...",Secretariat,2020-09-15,2021-09-14
1,Royal African Society,Secretariat From : 08/01/2021 To : 07/01/2022,"31,501-33,000",08/01/2021,21/04/2021,africa,"<Result () {'thing': 'Secretariat ', 'from': ...",Secretariat,2021-01-08,2022-01-07
2,Chatham House,Secretariat From : 29/01/2021 To : 28/01/2022,"9,001-10,500",29/01/2021,08/03/2021,angola,"<Result () {'thing': 'Secretariat ', 'from': ...",Secretariat,2021-01-29,2022-01-28
3,CalComms,Secretariat From : 10/09/2020 To : 09/09/2021,"13,501-15,000",10/09/2020,24/09/2020,canada-australia-new-zealand-and-the-uk-canzuk,"<Result () {'thing': 'Secretariat ', 'from': ...",Secretariat,2020-09-10,2021-09-09
4,Samir Takla,Samir Takla acts as the group's secretariat an...,"4,501-6,000",29/01/2021,15/02/2021,egypt,"<Result () {'thing': ""Samir Takla acts as the ...",Samir Takla acts as the group's secretariat an...,2021-01-29,2022-01-28


## Entity Extraction

We can trivially extract entities from a text field using the `spacy` package:

In [96]:
#%pip install --upgrade spacy
import spacy
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.9/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


For example, extract entities in the *Description* column:

In [105]:
get_entities = lambda desc: [f"{entity.label_} :: {entity.text}" for entity in nlp(desc).ents]

df['entities'] = df["Description"].apply(get_entities)
df.head()

,Source,Description,"Value £s In bands of £1,500",Received,Registered,gid,from_to,entities
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"<Result () {'thing': 'Secretariat ', 'from': ...","[ORG :: Secretariat, DATE :: 15/09/2020, DATE ..."
1,Royal African Society,Secretariat From : 08/01/2021 To : 07/01/2022,"31,501-33,000",08/01/2021,21/04/2021,africa,"<Result () {'thing': 'Secretariat ', 'from': ...",[ORG :: Secretariat]
2,Chatham House,Secretariat From : 29/01/2021 To : 28/01/2022,"9,001-10,500",29/01/2021,08/03/2021,angola,"<Result () {'thing': 'Secretariat ', 'from': ...","[ORG :: Secretariat, DATE :: 29/01/2021, DATE ..."
3,CalComms,Secretariat From : 10/09/2020 To : 09/09/2021,"13,501-15,000",10/09/2020,24/09/2020,canada-australia-new-zealand-and-the-uk-canzuk,"<Result () {'thing': 'Secretariat ', 'from': ...","[ORG :: Secretariat, DATE :: 10/09/2020]"
4,Samir Takla,Samir Takla acts as the group's secretariat an...,"4,501-6,000",29/01/2021,15/02/2021,egypt,"<Result () {'thing': ""Samir Takla acts as the ...","[PERSON :: Samir Takla, PERSON :: Carole Cosgr..."


We can explode these out into a long format dataframe:

In [109]:
df_long = df.explode('entities')
df_long.rename(columns={"entities":"entity"}, inplace=True)
df_long.head()

,Source,Description,"Value £s In bands of £1,500",Received,Registered,gid,from_to,entity
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"<Result () {'thing': 'Secretariat ', 'from': ...",ORG :: Secretariat
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"<Result () {'thing': 'Secretariat ', 'from': ...",DATE :: 15/09/2020
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"<Result () {'thing': 'Secretariat ', 'from': ...",DATE :: 14/09/2021
1,Royal African Society,Secretariat From : 08/01/2021 To : 07/01/2022,"31,501-33,000",08/01/2021,21/04/2021,africa,"<Result () {'thing': 'Secretariat ', 'from': ...",ORG :: Secretariat
2,Chatham House,Secretariat From : 29/01/2021 To : 28/01/2022,"9,001-10,500",29/01/2021,08/03/2021,angola,"<Result () {'thing': 'Secretariat ', 'from': ...",ORG :: Secretariat


And maybe make them easier to search on by type?

In [112]:
df_long[["entity_typ", "entity_value"]] = df_long["entity"].str.split(" :: ").apply(Series)
df_long.head()

,Source,Description,"Value £s In bands of £1,500",Received,Registered,gid,from_to,entity,entity_typ,entity_value
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"<Result () {'thing': 'Secretariat ', 'from': ...",ORG :: Secretariat,ORG,Secretariat
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"<Result () {'thing': 'Secretariat ', 'from': ...",DATE :: 15/09/2020,DATE,15/09/2020
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"<Result () {'thing': 'Secretariat ', 'from': ...",DATE :: 14/09/2021,DATE,14/09/2021
1,Royal African Society,Secretariat From : 08/01/2021 To : 07/01/2022,"31,501-33,000",08/01/2021,21/04/2021,africa,"<Result () {'thing': 'Secretariat ', 'from': ...",ORG :: Secretariat,ORG,Secretariat
2,Chatham House,Secretariat From : 29/01/2021 To : 28/01/2022,"9,001-10,500",29/01/2021,08/03/2021,angola,"<Result () {'thing': 'Secretariat ', 'from': ...",ORG :: Secretariat,ORG,Secretariat


What types of entity are there?

In [113]:
df_long['entity_typ'].value_counts()

# GPE - geo-political entity

ORG            579
DATE           428
PERSON          82
GPE             51
CARDINAL        27
NORP            26
FAC              4
EVENT            2
LOC              2
WORK_OF_ART      1
PRODUCT          1
Name: entity_typ, dtype: int64

In [118]:
df_long[df_long['entity_typ']=='ORG']["entity_value"].value_counts()

Secretariat               231
APPG                        6
Healthcomms Consulting      5
College Green Group         4
DevoConnect                 4
                         ... 
KPMG UK                     1
The Gym Group               1
EDF Energy                  1
EELGA                       1
Access Industry Forum       1
Name: entity_value, Length: 310, dtype: int64

## Extract Value Range

The value range would be more useful if it were split into "lower_value" and "upper_value" bands:

In [129]:
df[["lower_bound", "upper_bound"]] = df["Value £s  In bands of £1,500"].str.split("-").apply(Series)

df

,Source,Description,"Value £s In bands of £1,500",Received,Registered,gid,lower_bound,upper_bound
0,CalComms,Secretariat From : 15/09/2020 To : 14/09/2021,"13,501-15,000",15/09/2020,17/09/2020,afghanistan,"13,501","15,000"
1,Royal African Society,Secretariat From : 08/01/2021 To : 07/01/2022,"31,501-33,000",08/01/2021,21/04/2021,africa,"31,501","33,000"
2,Chatham House,Secretariat From : 29/01/2021 To : 28/01/2022,"9,001-10,500",29/01/2021,08/03/2021,angola,"9,001","10,500"
3,CalComms,Secretariat From : 10/09/2020 To : 09/09/2021,"13,501-15,000",10/09/2020,24/09/2020,canada-australia-new-zealand-and-the-uk-canzuk,"13,501","15,000"
4,Samir Takla,Samir Takla acts as the group's secretariat an...,"4,501-6,000",29/01/2021,15/02/2021,egypt,"4,501","6,000"
...,...,...,...,...,...,...,...,...
382,Association for Young People's Health,Secretariat From : 04/02/2021 To : 03/02/2022,"4,501-6,000",04/02/2021,07/06/2021,young-peoples-health,"4,501","6,000"
383,InterClimate Network,Secretariat From : 15/01/2021 To : 14/01/2022,"6,001-7,500",15/01/2021,07/07/2021,youth-action-against-climate-change,"6,001","7,500"
384,British Youth Council,Joint Secretariat From : 29/01/2021 To : 28/...,"1,501-3,000",29/01/2021,01/06/2021,youth-affairs,"1,501","3,000"
385,YMCA England and Wales,Joint Secretariat From : 29/01/2021 To : 28/...,"1,501-3,000",29/01/2021,01/06/2021,youth-affairs,"1,501","3,000"
